<a href="https://colab.research.google.com/github/dzakywd/talent-matching-intelligence-system/blob/main/talentmatch_00_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Read the csv file that need to be cleaned
employees = pd.read_csv('employees.csv')
psych = pd.read_csv('profiles_psych.csv')
papi = pd.read_csv('papi_scores.csv')
strengths = pd.read_csv('strengths.csv')
performance = pd.read_csv('performance_yearly.csv')
competencies = pd.read_csv('competencies_yearly.csv')

dfs = {
    "employees": employees,
    "psych": psych,
    "papi": papi,
    "strengths": strengths,
    "performance": performance,
    "competencies": competencies
}

### Column Standardization

In [58]:
# Column standardization
def clean_columns(df):
  df.columns = (df.columns
                .str.lower()
                .str.strip()
                .str.replace(' ', '_')
                .str.replace('-', '_')
                .str.replace('.', '_'))
  return df

for name,df in dfs.items():
  dfs[name] = clean_columns(df)

employees = dfs["employees"]
psych = dfs["psych"]
papi = dfs["papi"]
strengths = dfs["strengths"]
performance = dfs["performance"]
competencies = dfs["competencies"]

### Numeric Coercion

In [59]:
# Numeric Coercion (make sure all the data in numeric column is numeric)
numeric_targets = [
    "years_of_service_months", "pauli", "faxtor", "iq", "gtq", "tiki",
    "score", "rating", "year",
]

for col in numeric_targets:
  if col in employees.columns:
    employees[col] = pd.to_numeric(employees[col], errors="coerce")
  if col in performance.columns:
    performance[col] = pd.to_numeric(performance[col], errors="coerce")
  if col in competencies.columns:
    competencies[col] = pd.to_numeric(competencies[col], errors="coerce")
  if col in papi.columns:
    papi[col] = pd.to_numeric(papi[col], errors="coerce")
  if col in psych.columns:
    psych[col] = pd.to_numeric(psych[col], errors="coerce")
  if col in strengths.columns:
    strengths[col] = pd.to_numeric(strengths[col], errors="coerce")

### Object Cleaning

In [60]:
import re

# 1) Valid data
valid_mbti = {
    "INTJ","INTP","ENTJ","ENTP",
    "INFJ","INFP","ENFJ","ENFP",
    "ISTJ","ISFJ","ESTJ","ESFJ",
    "ISTP","ISFP","ESTP","ESFP"
}

valid_disc = {
    "DI","ID","DS","SD","DC","CD",
    "IS","SI","IC","CI","SC","CS"
}

valid_strengths = {
    "Achiever", "Activator", "Adaptability", "Analytical", "Arranger",
    "Belief", "Command", "Communication", "Competition", "Connectedness",
    "Consistency", "Context", "Deliberative", "Developer", "Discipline",
    "Empathy", "Focus", "Futuristic", "Harmony", "Ideation", "Includer",
    "Individualization", "Input", "Intellection", "Learner", "Maximizer",
    "Positivity", "Relator", "Responsibility", "Restorative",
    "Self-Assurance", "Significance", "Strategic", "Woo"
}

valid_pillar = {
    "GDR", "CEX", "IDS", "QDD", "STO", "SEA", "VCU", "LIE", "FTC", "CSI"
}

valid_papi = {f"papi_{chr(i)}" for i in range(ord('a'), ord('z')+1)}

# 2) Object type data cleaning

def clean_object_columns(df):
    df = df.copy()
    obj_cols = df.select_dtypes(include="object").columns
    for col in obj_cols:

        # Basic cleaning
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
        )

        # MBTI
        if col.lower() == "mbti":
            df[col] = df[col].str.upper()
            df[col] = df[col].apply(lambda x: x if x in valid_mbti else np.nan)

        # DISC
        elif col.lower() == "disc":
            df[col] = df[col].str.upper()
            df[col] = df[col].apply(lambda x: x if x in valid_disc else np.nan)

        # Strength themes
        elif "theme" in col.lower():
            df[col] = df[col].str.title()
            df[col] = df[col].apply(
                lambda x: x if x in valid_strengths else np.nan
            )

        # PILLAR CODE
        elif col.lower() == "pillar_code":
            df[col] = df[col].str.upper().str.replace(" ", "")
            df[col] = df[col].apply(lambda x: x if x in valid_pillar else np.nan)

        # PAPI scale CODE
        elif col.lower() == "scale_code":
            df[col] = (
                df[col]
                .str.lower()
                .str.replace(" ", "")
                .str.replace("-", "")
            )
            # Valid format
            df[col] = df[col].apply(lambda x: x if x in valid_papi else np.nan)

    return df

In [61]:
# Start cleaning
for name, df in dfs.items():
    dfs[name] = clean_object_columns(df)

employees = dfs["employees"]
psych = dfs["psych"]
papi = dfs["papi"]
strengths = dfs["strengths"]
performance = dfs["performance"]
competencies = dfs["competencies"]

### Numeric Cleaning

In [62]:
# Quick Audit Function
def audit(df, name):
  print(f"===== {name.upper()} =====")
  print(df.head(), "\n")
  print(df.info(), "\n")
  print("Missing values:\n", df.isna().sum(), "\n")
  print("Duplicates:", df.duplicated().sum(), "\n")
  print("---- UNIQUE VALUES ----")
  for col in df.columns:
      if df[col].dtype == 'object':
          print(f"{col}: {df[col].unique()[:10]} ... ({df[col].nunique()} unique)")
      else:
          # for numeric: show range instead of full unique list
          print(f"{col}: min={df[col].min()}, max={df[col].max()}, unique={df[col].nunique()}")
  print("="*60, "\n")

for name, df in dfs.items():
  audit(df, name)

===== EMPLOYEES =====
  employee_id                  fullname     nip  company_id  area_id  \
0   EMP100000            Rendra Pratama  806137           1        4   
1   EMP100001            Wulan Setiawan  476388           4        4   
2   EMP100002  Julia Jatmiko Situmorang  941921           1        3   
3   EMP100003                 Oka Halim  751615           1        3   
4   EMP100004               Dwi Pratama  443809           3        4   

   position_id  department_id  division_id  directorate_id  grade_id  \
0            1              6            3               3         2   
1            4              5            5               3         1   
2            5              1            1               3         1   
3            4              2            1               1         2   
4            6              5            3               3         1   

   education_id  major_id  years_of_service_months  
0             3         1                       64  
1     

In [63]:
# Data range validation -> Check for anomalies
def validate_ranges(df, column, valid_min=None, valid_max=None, valid_set=None):
    if valid_set:
        invalid = ~df[column].isin(valid_set)
    else:
        invalid = (df[column] < valid_min) | (df[column] > valid_max)

    invalid_count = invalid.sum()
    if invalid_count > 0:
        print(f"⚠️  INVALID VALUES in {column}: {invalid_count} rows")
        print(df.loc[invalid, column].value_counts().head(), "\n")
    else:
        print(f"✓ {column} OK (no invalid values)\n")

print("=========== VALIDATION START ===========\n")

# PSYCHOMETRIC TESTS
print("PSYCH")

# Pauli (~20-100)
validate_ranges(psych, "pauli", valid_min=20, valid_max=100)

# Faxtor (~20-100)
validate_ranges(psych, "faxtor", valid_min=20, valid_max=100)

# IQ (~80-140)
validate_ranges(psych, "iq", valid_min=80, valid_max=140)

# GTQ (usually 0–120, check for negatives)
validate_ranges(psych, "gtq", valid_min=0, valid_max=120)

# tiki (must be between 1-10)
validate_ranges(psych, "tiki", valid_min=1, valid_max=10)


# PAPI (must be between 1-9)
print("PAPI")
validate_ranges(papi, "score", valid_min=1, valid_max=9)


# STRENGTHS (must be between 1–14)
print("STRENGTHS")
validate_ranges(strengths, "rank", valid_min=1, valid_max=14)


# PERFORMANCE (must be between 1–5)
print("PERFORMANCE")
validate_ranges(performance, "rating", valid_min=1, valid_max=5)


# COMPETENCIES (must be between 1–5)
print("COMPETENCIES")
validate_ranges(competencies, "score", valid_min=1, valid_max=5)


print("=========== VALIDATION END ===========")

=========== VALIDATION START ===========

PSYCH
✓ pauli OK (no invalid values)

✓ faxtor OK (no invalid values)

✓ iq OK (no invalid values)

✓ gtq OK (no invalid values)

✓ tiki OK (no invalid values)

PAPI
✓ score OK (no invalid values)

STRENGTHS
✓ rank OK (no invalid values)

PERFORMANCE
⚠️  INVALID VALUES in rating: 35 rows
rating
0.0     15
6.0     13
99.0     7
Name: count, dtype: int64 

COMPETENCIES
⚠️  INVALID VALUES in score: 499 rows
score
99.0    171
6.0     165
0.0     163
Name: count, dtype: int64 

=========== VALIDATION END ===========


In [64]:
# Replace invalid value wiyh NaN
performance.loc[
    (performance["rating"]<1)|(performance["rating"]>5), "rating"
] = np.nan

competencies.loc[
    (competencies["score"]<1)|(competencies["score"]>5), "score"
] = np.nan

In [65]:
# Re-run validation
print("=========== VALIDATION START ===========\n")

# PERFORMANCE (must be between 1–5)
print("PERFORMANCE")
validate_ranges(performance, "rating", valid_min=1, valid_max=5)

# COMPETENCIES (must be between 1–5)
print("COMPETENCIES")
validate_ranges(competencies, "score", valid_min=1, valid_max=5)

print("=========== VALIDATION END ===========")

=========== VALIDATION START ===========

PERFORMANCE
✓ rating OK (no invalid values)

COMPETENCIES
✓ score OK (no invalid values)

=========== VALIDATION END ===========


In [66]:
# Fill the missing value in psych table with their median
for col in ["iq", "faxtor", "gtq", "tiki"]:
    psych[col] = psych[col].fillna(psych[col].median())

# Fill the mising value of score data in papi scores table using their median grouped by scale code
papi["score"] = papi.groupby("scale_code")["score"].transform(
    lambda x: x.fillna(x.median())
)

# Drop all the row in performance table that doesn't have "rating" value
performance = performance.dropna(subset=["rating"])

# Fill the missing value of score data in competencies table using their median grouped by pillar code and year
competencies["score"] = competencies.groupby(["pillar_code", "year"])["score"].transform(
    lambda x: x.fillna(x.median())
)

In [67]:
# Check employee_id consistency

core_ids = set(employees["employee_id"])

print("Psych missing:", set(psych["employee_id"]) - core_ids)
print("PAPI missing:", set(papi["employee_id"]) - core_ids)
print("Strenghths missing:", set(strengths["employee_id"]) - core_ids)
print("Performance missing:", set(performance["employee_id"]) - core_ids)
print("Competencies missing:", set(competencies["employee_id"]) - core_ids)

Psych missing: set()
PAPI missing: set()
Strenghths missing: set()
Performance missing: set()
Competencies missing: set()


In [68]:
dfs["employees"] = employees
dfs["psych"] = psych
dfs["papi"] = papi
dfs["strengths"] = strengths
dfs["performance"] = performance
dfs["competencies"] = competencies

In [69]:
# Re-run audit
for name, df in dfs.items():
  audit(df, name)

===== EMPLOYEES =====
  employee_id                  fullname     nip  company_id  area_id  \
0   EMP100000            Rendra Pratama  806137           1        4   
1   EMP100001            Wulan Setiawan  476388           4        4   
2   EMP100002  Julia Jatmiko Situmorang  941921           1        3   
3   EMP100003                 Oka Halim  751615           1        3   
4   EMP100004               Dwi Pratama  443809           3        4   

   position_id  department_id  division_id  directorate_id  grade_id  \
0            1              6            3               3         2   
1            4              5            5               3         1   
2            5              1            1               3         1   
3            4              2            1               1         2   
4            6              5            3               3         1   

   education_id  major_id  years_of_service_months  
0             3         1                       64  
1     

### Save The Cleaned Version for Download

In [70]:
import os
os.makedirs("data_clean", exist_ok=True)

In [71]:
employees.to_csv("data_clean/employees_clean.csv", index=False)
performance.to_csv("data_clean/performance_clean.csv", index=False)
competencies.to_csv("data_clean/competencies_clean.csv", index=False)
papi.to_csv("data_clean/papi_clean.csv", index=False)
psych.to_csv("data_clean/psych_clean.csv", index=False)
strengths.to_csv("data_clean/strengths_clean.csv", index=False)

In [72]:
papi

,employee_id,scale_code,score
0,EMP100000,papi_n,2.0
1,EMP100001,papi_n,8.0
2,EMP100002,papi_n,9.0
3,EMP100003,papi_n,9.0
4,EMP100004,papi_n,7.0
...,...,...,...
40195,DUP1942,papi_w,8.0
40196,DUP2096,papi_w,3.0
40197,DUP4177,papi_w,1.0
40198,DUP9801,papi_w,5.0
